# Simulation 2. Wahlgang Ständerat Solothurn 2023

Eingelegte Wahlzettel 2019: 39,3 Prozent der Stimmberechtigten: https://sostimmt.so.ch/gui/layout/2019/30

Wir rechnen mit den fiktiven Wählenden pro Partei. Auf diese Zahl kommt man, wenn man die gültigen Wahlzettel den Parteistärken entsprechend auf die Parteien aufteilt. Zusätzlich betrachten wir die Abstinenten. Diese definieren wir der Einfachheit halber inklusive leerer und ungültiger Wahlzettel. Diese machen im Kanton Solothurn zusammen 1,18 Prozent der Wahlberechtigten aus.

In [1]:
import requests
import pandas as pd
import numpy as np
from my_wahl_settings import parteien_zuordner

In [2]:
df_partei = pd.read_csv(parteien_zuordner)

Import Wahlberechtigte

In [3]:
url_be = 'https://ogd-static.voteinfo-app.ch/v4/ogd/sd-t-17.02-NRW2023-wahlbeteiligung.json'

r_be = requests.get(url_be)

r_be = r_be.json()

df_berechtigte = pd.DataFrame(r_be['level_kantone'])

gueltige_wahlzettel = df_berechtigte[df_berechtigte['kanton_nummer'] == 11]['gueltige_wahlzettel'].values[0]

wahlberechtigte = df_berechtigte[df_berechtigte['kanton_nummer'] == 11]['wahlberechtigte'].values[0]

Wie gross ist der Anteil leerer und ungültiger Wahlzettel?

In [4]:
df_berechtigte['leer_ungültig_pct'] = (df_berechtigte['leere_wahlzettel'] + df_berechtigte['ungueltige_wahlzettel']) / df_berechtigte['wahlberechtigte'] * 100

Import Wahldaten

In [5]:
url = 'https://ogd-static.voteinfo-app.ch/v4/ogd/sd-t-17.02-NRW2023-parteien.json'

r = requests.get(url)

r = r.json()

df = pd.DataFrame(r['level_kantone'])

df = df[df['kanton_nummer'] == 11].copy()

df = df_partei[['partei_id', 'partei_kurz']].merge(df, on='partei_id', how='right')

df = df[['partei_kurz', 'fiktive_waehlende']].copy()

Abstinente dazu fügen (Nichtwähler plus leere plus ungültige)

In [6]:
fiktive_abwesende = wahlberechtigte - gueltige_wahlzettel

abstinenten_dict = {'partei_kurz': ['Abstinente'], 'fiktive_waehlende': [fiktive_abwesende]}

df = pd.concat([df, pd.DataFrame(abstinenten_dict)])

Die SD hat keine Wähleranteile, darum schliessen wir sie aus.

In [7]:
df = df[df['partei_kurz'] != 'SD'].copy()

df.set_index('partei_kurz', inplace=True)

Kontrolle: Haben wir gleich viele fiktive Wählende wie Wahlberechtigte?

In [8]:
wahlberechtigte == df['fiktive_waehlende'].sum()

True

Szenario-Daten hinzufügen. Es handelt sich um Prozentwerte, entlang denen die fiktiven Wählenden pro Partei an die Kandidaten verteilt werden. Es müssen immer 100 Prozent sein.

In [9]:
df_szenario = pd.read_excel('szenario.xlsx', index_col='Partei')

df = df.join(df_szenario)

In [10]:
len(df_szenario.sum(axis=1)[df_szenario.sum(axis=1) == 100]) == len(df_szenario)

True

**Funktion zur Berechnung der Wähleranteile**

Wir runden die Stimmenzahlen auf den Hunderter, da wir mit groben Annahmen arbeiten und Bis auf den Einer genaue Zahlen darum nicht plausibel wären.

In [11]:
def ergebnis_rechner(df):
        
    df_func = df.copy()
    
    #Wir berechnen die Wähler pro Kandidat/in
    df_func['wähler_imark'] = df_func['fiktive_waehlende'] * (df_func['Imark'] / 100)
    df_func['wähler_roth'] = df_func['fiktive_waehlende'] * (df_func['Roth'] / 100)
    df_func['wähler_enthaltung'] = df_func['fiktive_waehlende'] * (df_func['Enthaltung'] / 100)

    df_func = df_func.replace(np.inf, 0)
    
    first_check = wahlberechtigte == round(df_func['wähler_imark'].sum() + df_func['wähler_roth'].sum() + df_func['wähler_enthaltung'].sum(),4)
       
    if first_check != True:
        raise ValueError('Mit der Wählerzuteilung stimmt etwas nicht.')
    
    summe_imark_roth = df_func['wähler_imark'].sum() + df_func['wähler_roth'].sum()
    
    #Wähleranteile
    wähleranteil_imark = df_func['wähler_imark'].sum() / summe_imark_roth * 100
    wähleranteil_roth = df_func['wähler_roth'].sum() / summe_imark_roth * 100
    
    wähler_imark = df_func['wähler_imark'].sum()
    wähler_roth = df_func['wähler_roth'].sum()
    
    kandi_check = wähleranteil_imark + wähleranteil_roth
    
    if kandi_check < 99.9 or kandi_check > 100.1:
        raise ValueError('Irgendetwas stimmt mit den Wähleranteilen von imark oder roth nicht.')
        
    #Enthaltung / Beteiligung
    anteil_enthaltung = df_func['wähler_enthaltung'].sum() / wahlberechtigte * 100
    
    wahlbeteiligung = 100 - anteil_enthaltung
    
    if wahlbeteiligung > 100 or wahlbeteiligung < 10:
        raise ValueError('Irgendetwas stimmt mit der Wahlbeteiligung nicht.')
        
    result_dict = {
        'wähleranteil_imark': [round(wähleranteil_imark, 1)],
        'wähleranteil_roth': [round(wähleranteil_roth, 1)],
        'wähler_imark': [round(wähler_imark, -2)],
        'wähler_roth': [round(wähler_roth, -2)],
        'anteil_enthaltung': [round(anteil_enthaltung, 1)],
        'wahlbeteiligung': [round(wahlbeteiligung, 1)]
    }
    
    df_results = pd.DataFrame(result_dict)
    
    return df_results

**Berechnungen - Szenario 0**

In [12]:
ergebnis_rechner(df)

,wähleranteil_imark,wähleranteil_roth,wähler_imark,wähler_roth,anteil_enthaltung,wahlbeteiligung
0,53.2,46.8,37900.0,33400.0,61.1,38.9


**Berechnungen - Szenario 1**

Ausgangsfrage: Was passiert bei einer hohen Wahlbeteiligung?

In [13]:
df_szenario1 = df.copy()

df_szenario_extra = pd.read_excel('szenario_10.xlsx')

df_szenario1 = df_szenario1[['fiktive_waehlende']].join(df_szenario_extra.set_index('Partei'))

df_ergebnis_temp1 = ergebnis_rechner(df_szenario1)

In [14]:
df_ergebnis_temp1

,wähleranteil_imark,wähleranteil_roth,wähler_imark,wähler_roth,anteil_enthaltung,wahlbeteiligung
0,53.0,47.0,42700.0,37900.0,56.1,43.9


**Berechnungen - Szenario 2**

Ausgangsfrage: Wie fällt das Ergebnis aus, wenn die teilnehmenden Mitte-Wählenden zu 80 Prozent für Roth stimmen?

In [15]:
df_szenario2 = df.copy()

df_szenario2.loc['Mitte', 'Roth'] = 56
df_szenario2.loc['Mitte', 'Imark'] = 14

ergebnis_rechner(df_szenario2)

,wähleranteil_imark,wähleranteil_roth,wähler_imark,wähler_roth,anteil_enthaltung,wahlbeteiligung
0,49.7,50.3,35500.0,35800.0,61.1,38.9


**Berechnungen - Szenario 3**

Ausgangsfrage: Bei der entzweiten FDP holt Roth 1/3 der Stimmen.

In [16]:
df_szenario3 = df.copy()

df_szenario3.loc['FDP', 'Imark'] = 47
df_szenario3.loc['FDP', 'Roth'] = 23

ergebnis_rechner(df_szenario3)

,wähleranteil_imark,wähleranteil_roth,wähler_imark,wähler_roth,anteil_enthaltung,wahlbeteiligung
0,50.4,49.6,36000.0,35300.0,61.1,38.9


**Berechnungen - Szenario 4**

Ausgangsfrage: Was passiert, wenn die SVP-Wählerschaft ein bisschen wahlabstinent ist, die FDP und Mitte deutlicher?

In [17]:
df_szenario4 = df.copy()

df_szenario_extra4 = pd.read_excel('szenario_rechte_wahlabstinenz.xlsx')

df_szenario4 = df_szenario4[['fiktive_waehlende']].join(df_szenario_extra4.set_index('Partei'))

ergebnis_rechner(df_szenario4)

,wähleranteil_imark,wähleranteil_roth,wähler_imark,wähler_roth,anteil_enthaltung,wahlbeteiligung
0,50.0,50.0,32900.0,33000.0,64.1,35.9
